# Frequentist uncertainty vs. Bayesian uncertainty analysis
*Mark Bakker, TU Delft & Raoul Collenteur, Eawag, February, 2025*

In this notebook, the fit and uncertainty are compared for `pastas` models solved with least squares (frequentist uncertainty) and with MCMC (Bayesian uncertainty). 
Besides Pastas the following Python Packages have to be installed to run this notebook:

- [emcee](https://emcee.readthedocs.io)
- [corner](https://corner.readthedocs.io)

<div class="alert alert-warning">
<b>Note:</b>
The EmceeSolver is still an experimental feature and some of the arguments may change in before official release. We welcome testing and feedback on this new feature!
</div>

In [ ]:
import corner
import emcee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pastas as ps

ps.set_log_level("ERROR")
ps.show_versions()

## 1. A 'regular' Pastas Model
The first step is to create a Pastas Model with a linear `RechargeModel` and a `Gamma` response function to simulate the effect of precipitation and evaporation on the heads. The AR1 noise model is used. We first estimate the model parameters using the standard least-squares approach.

In [ ]:
head = pd.read_csv(
    "data/B32C0639001.csv", parse_dates=["date"], index_col="date"
).squeeze()
head = head["1990":]  # use data from 1990 on for this example

evap = pd.read_csv("data/evap_260.csv", index_col=0, parse_dates=[0]).squeeze()
rain = pd.read_csv("data/rain_260.csv", index_col=0, parse_dates=[0]).squeeze()

ml1 = ps.Model(head)
ml1.add_noisemodel(ps.ArNoiseModel())

rm = ps.RechargeModel(
    rain, evap, recharge=ps.rch.Linear(), rfunc=ps.Gamma(), name="rch"
)
ml1.add_stressmodel(rm)

ml1.solve()

ax = ml1.plots.results(figsize=(10, 4))

The diagnostics show that the noise meets the statistical requirements for uncertainty analysis reasonably well. 

In [ ]:
ml1.plots.diagnostics();

The estimated least squares parameters and standard errors are stored for later reference

In [ ]:
ls_params = ml1.parameters[["optimal", "stderr"]].copy()
ls_params.rename(columns={"optimal": "ls_opt", "stderr": "ls_sig"}, inplace=True)
ls_params

In [ ]:
# Compute prediction interval Pastas
pi = ml1.solver.prediction_interval(n=1000)
ax = ml1.plot(figsize=(10, 3))
ax.fill_between(pi.index, pi.iloc[:, 0], pi.iloc[:, 1], color="lightgray")
ax.legend(["Observations", "Simulation", "95% Prediction interval"], ncol=3, loc=2)
pi_pasta = np.mean(pi[0.975] - pi[0.025])
print(f"Mean prediction interval width: {pi_pasta:.3f} m")
print(f"Prediction interval coverage probability: {ps.stats.picp(head, pi): .3f}")

## 2. Use the EmceeSolver

We will now use MCMC to estimate the model parameters and their uncertainties. The `EmceeSolve` solver wraps the [Emcee](https://lmfit.github.io/lmfit-py/fitting.html#lmfit.minimizer.Minimizer.emcee) package, which implements different versions of MCMC. A good understanding of Emcee helps when using this solver, so it comes recommended to check out their documentation as well.

We start by making a `pastas` model with a linear recharge model and a Gamma response function. No noise model is added, as this is taken care of in the likelihood function. The model is solved using the regular solve (least squares) to have a good estimate of the starting values of the parameters.

In [ ]:
ml2 = ps.Model(head)
rm = ps.RechargeModel(
    rain, evap, recharge=ps.rch.Linear(), rfunc=ps.Gamma(), name="rch"
)
ml2.add_stressmodel(rm)
ml2.solve()

To set up the `EmceeSolve` solver, a number of decisions need to be made:

- Select the priors of the parameters
- Select a (log) likelihood function
- Selecte the number of steps and thinning
  
### 2a. Priors

The first step is to select the priors of the parameters. This is done by using the `ml.set_parameter` method and the `dist` argument (from distribution). Any distribution from `scipy.stats` can be chosen [url](https://docs.scipy.org/doc/scipy/tutorial/stats/continuous.html), for example `uniform`, `norm`, or `lognorm`. Here, we select normal distributions for the priors. Currently, `pastas` will use the `initial` value of the parameter for the `loc` argument of the distribution (e.g., the mean of a normal distribution), and the `stderr` as the `scale` argument (e.g., the standard deviation of a normal distribution). Only for the parameters with a `uniform` distribution, the `pmin` and `pmax` values are used to determine a uniform prior. By default, all parameters are assigned a `uniform` prior.

<div class="alert alert-warning">
<b>Note:</b>
This means that either the `pmin` and `pmax` should be set for uniform distributions, or the `stderr` for any other distribution. That is why in this example model was first solved using LeastSquares, in order to obtain estimates for the `stderr`. In practice, these could also be set based on expert judgement or information about the parameters.
</div>

In [ ]:
# Set the initial parameters to a normal distribution
ml2.parameters["initial"] = ml2.parameters[
    "optimal"
]  # set initial value to the optimal from least squares for good starting point
ml2.parameters["stderr"] = (
    2 * ml2.parameters["stderr"]
)  # this column is used (for now) to set the scale of the normal distribution

for name in ml2.parameters.index:
    ml2.set_parameter(
        name,
        dist="norm",
    )

ml2.parameters



### 2b. Create the solver instance

The next step is to create an instance of the `EmceeSolve` solver class. At this stage all the settings need to be provided on how the Ensemble Sampler is created (https://emcee.readthedocs.io/en/stable/user/sampler/). Important settings are the `nwalkers`, the `moves`, the `objective_function`. More advanced options are to parallelize the MCMC algorithm (`parallel=True`), and to set a backend to store the results. Here's an example:

In [ ]:
# Choose the objective function
ln_prob = ps.objfunc.GaussianLikelihoodAr1()

# Create the EmceeSolver with some settings
s = ps.EmceeSolve(
    nwalkers=20,
    moves=emcee.moves.DEMove(),
    objective_function=ln_prob,
    progress_bar=True,
    parallel=False,
)

In the above code we created an `EmceeSolve` instance with 20 walkers, which take steps according to the `DEMove` move algorithm (see Emcee docs), and a Gaussian likelihood function that assumes AR1 correlated errors. Different objective functions are available, see the Pastas documentation on the different options. 

Depending on the likelihood function, a number of additional parameters need to be inferred. These parameters are not added to the Pastas Model instance, but are available from the solver object. Using the `set_parameter` method of the solver, these parameters can be changed. In this example where we use the `GaussianLikelihoodAr1` function the sigma and theta are estimated; the unknown standard deviation of the errors and the autoregressive parameter.

In [ ]:
s.parameters

In [ ]:
sigsq = ml1.noise().std() ** 2
s.set_parameter("ln_var", initial=sigsq, vary=True)
s.parameters.loc["ln_var", "stderr"] = stderr = sigsq / 8
s.parameters

### 2c. Run the solver and solve the model

After setting the parameters and creating a EmceeSolve solver instance we are now ready to run the MCMC analysis. We can do this by running `ml.solve`. We can pass the same parameters that we normally provide to this method (e.g., `tmin` or `fit_constant`). Here we use the initial parameters from our least-square solve, and do not fit a noise model, because we take autocorrelated errors into account through the likelihood function. 

All the arguments that are not used by `ml.solve`, for example `steps` and `tune`, are passed on to the `run_mcmc` method from the sampler (see Emcee docs). The most important is the `steps` argument, that determines how many steps each of the walkers takes.

In [ ]:
# Use the solver to run MCMC
ml2.solve(
    solver=s,
    initial=False,
    tmin="1990",
    steps=1000,
    tune=True,
    report=False,
)

## 3. Posterior parameter distributions

The results from the MCMC analysis are stored in the `sampler` object, accessible through `ml.solver.sampler` variable. The object `ml.solver.sampler.flatchain` contains a Pandas DataFrame with $n$ the parameter samples, where $n$ is calculated as follows:

$n = \frac{\left(\text{steps}-\text{burn}\right)\cdot\text{nwalkers}}{\text{thin}} $

## Corner.py
Corner is a simple but great python package that makes creating corner graphs easy. A couple of lines of code suffice to create a plot of the parameter distributions and the covariances between the parameters.

In [ ]:
# Corner plot of the results
fig = plt.figure(figsize=(8, 8))

labels = list(ml2.parameters.index[ml2.parameters.vary]) + list(
    ml2.solver.parameters.index[ml2.solver.parameters.vary]
)
labels = [label.split("_")[1] for label in labels]

best = list(ml2.parameters[ml2.parameters.vary].optimal) + list(
    ml2.solver.parameters[ml2.solver.parameters.vary].optimal
)

axes = corner.corner(
    ml2.solver.sampler.get_chain(flat=True, discard=500),
    quantiles=[0.025, 0.5, 0.975],
    labelpad=0.1,
    show_titles=True,
    title_kwargs=dict(fontsize=10),
    label_kwargs=dict(fontsize=10),
    max_n_ticks=3,
    fig=fig,
    labels=labels,
    truths=best,
)

plt.show()

## 4. The trace shows when MCMC converges
The walkers take steps in different directions for each step. It is expected that after a number of steps, the direction of the step becomes random, as a sign that an optimum has been found. This can be checked by looking at the autocorrelation, which should be insignificant after a number of steps. Below we just show how to obtain the different chains, the interpretation of which is outside the scope of this notebook.

In [ ]:
fig, axes = plt.subplots(len(labels), figsize=(10, 7), sharex=True)

samples = ml2.solver.sampler.get_chain(flat=True)
for i in range(len(labels)):
    ax = axes[i]
    ax.plot(samples[:, i], "k", alpha=0.5)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")

In [ ]:
mcn_params = pd.DataFrame(index=ls_params.index, columns=["mcn_opt", "mcn_sig"])
params = ml2.solver.sampler.get_chain(
    flat=True, discard=500
)  # discard first 500 of every chain
for iparam in range(params.shape[1] - 1):
    mcn_params.iloc[iparam] = np.median(params[:, iparam]), np.std(params[:, iparam])
mean_time_diff = head.index.to_series().diff().mean().total_seconds() / 86400
mcn_params.loc["noise_alpha", "mcn_opt"] = -mean_time_diff / np.log(
    np.median(params[:, -1])
)
mcn_params.loc["noise_alpha", "mcn_sig"] = -mean_time_diff / np.log(
    np.std(params[:, -1])
)

In [ ]:
pd.concat((ls_params, mcn_params), axis=1)

## Repeat with uniform priors
Set more or less uninformative uniform priors. Now also include $\sigma^2$. 

In [ ]:
ml3 = ps.Model(head)
rm = ps.RechargeModel(
    rain, evap, recharge=ps.rch.Linear(), rfunc=ps.Gamma(), name="rch"
)
ml3.add_stressmodel(rm)
ml3.solve(report=False)

Uniform prior selected from 0.25 till 4 times the optimal values

In [ ]:
# Set the initial parameters to a normal distribution
ml3.parameters["initial"] = ml3.parameters[
    "optimal"
]  # set initial value to the optimal from least squares for good starting point
for name in ml3.parameters.index:
    if ml3.parameters.loc[name, "optimal"] > 0:
        ml3.set_parameter(
            name,
            dist="uniform",
            pmin=0.25 * ml3.parameters.loc[name, "optimal"],
            pmax=4 * ml3.parameters.loc[name, "optimal"],
        )
    else:
        ml3.set_parameter(
            name,
            dist="uniform",
            pmin=4 * ml3.parameters.loc[name, "optimal"],
            pmax=0.25 * ml3.parameters.loc[name, "optimal"],
        )

ml3.parameters

In [ ]:
# Choose the objective function
ln_prob = ps.objfunc.GaussianLikelihoodAr1()

# Create the EmceeSolver with some settings
s = ps.EmceeSolve(
    nwalkers=20,
    moves=emcee.moves.DEMove(),
    objective_function=ln_prob,
    progress_bar=True,
    parallel=False,
)

s.parameters.loc["ln_var", "initial"] = 0.05**2
s.parameters.loc["ln_var", "pmin"] = 0.05**2 / 4
s.parameters.loc["ln_var", "pmax"] = 4 * 0.05**2

# Use the solver to run MCMC
ml3.solve(
    solver=s,
    initial=False,
    tmin="1990",
    steps=1000,
    tune=True,
    report=False,
)

In [ ]:
s.parameters

In [ ]:
# Corner plot of the results
fig = plt.figure(figsize=(8, 8))

labels = list(ml3.parameters.index[ml3.parameters.vary]) + list(
    ml3.solver.parameters.index[ml3.solver.parameters.vary]
)
labels = [label.split("_")[1] for label in labels]

best = list(ml3.parameters[ml3.parameters.vary].optimal) + list(
    ml3.solver.parameters[ml3.solver.parameters.vary].optimal
)

axes = corner.corner(
    ml3.solver.sampler.get_chain(flat=True, discard=500),
    quantiles=[0.025, 0.5, 0.975],
    labelpad=0.1,
    show_titles=True,
    title_kwargs=dict(fontsize=10),
    label_kwargs=dict(fontsize=10),
    max_n_ticks=3,
    fig=fig,
    labels=labels,
    truths=best,
)

plt.show()

In [ ]:
fig, axes = plt.subplots(len(labels), figsize=(10, 7), sharex=True)

samples = ml3.solver.sampler.get_chain(flat=True)
for i in range(len(labels)):
    ax = axes[i]
    ax.plot(samples[:, i], "k", alpha=0.5)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")

In [ ]:
mcu_params = pd.DataFrame(index=ls_params.index, columns=["mcu_opt", "mcu_sig"])
params = ml3.solver.sampler.get_chain(
    flat=True, discard=500
)  # discard first 500 of every chain
for iparam in range(params.shape[1] - 1):
    mcu_params.iloc[iparam] = np.median(params[:, iparam]), np.std(params[:, iparam])
mean_time_diff = head.index.to_series().diff().mean().total_seconds() / 86400
mcu_params.loc["noise_alpha", "mcu_opt"] = -mean_time_diff / np.log(
    np.median(params[:, -1])
)
mcu_params.loc["noise_alpha", "mcu_sig"] = -mean_time_diff / np.log(
    np.std(params[:, -1])
)

In [ ]:
pd.concat((ls_params, mcn_params, mcu_params), axis=1)

## 5. Compute prediction interval

In [ ]:
nobs = len(head)
params = ml3.solver.sampler.get_chain(flat=True, discard=500)
sim = {}
# compute for 1000 random samples of chain
np.random.seed(1)
for i in np.random.choice(np.arange(10000), size=1000, replace=False):
    h = ml3.simulate(p=params[i, :-2])
    res = ml3.residuals(p=params[i, :-2])
    h += np.random.normal(loc=0, scale=np.std(res), size=len(h))
    sim[i] = h
simdf = pd.DataFrame.from_dict(sim, orient="columns", dtype=float)
alpha = 0.05
q = [alpha / 2, 1 - alpha / 2]
pi = simdf.quantile(q, axis=1).transpose()
pimean = np.mean(pi[0.975] - pi[0.025])
print(f"prediction interval emcee with uniform priors: {pimean:.3f} m")
print(f"PICP: {ps.stats.picp(head, pi):.3f}")

For this example, the prediction interval is dominated by the residuals not by the uncertainty of the parameters. In the code cell below, the parameter uncertainty is not included: the coverage only changes slightly and is mostly affected by the difference in randomly drawing residuals.

In [ ]:
logprob = ml3.solver.sampler.compute_log_prob(
    ml3.solver.sampler.get_chain(flat=True, discard=500)
)[0]
imax = np.argmax(logprob)  # parameter set with larges likelihood
#
nobs = len(head)
params = ml3.solver.sampler.get_chain(flat=True, discard=500)
sim = {}
# compute for 1000 random samples of residuals, but one parameter set
h = ml3.simulate(p=params[imax, :-2])
res = ml3.residuals(p=params[imax, :-2])
np.random.seed(1)
for i in range(1000):
    sim[i] = h + np.random.normal(loc=0, scale=np.std(res), size=len(h))
simdf = pd.DataFrame.from_dict(sim, orient="columns", dtype=float)
alpha = 0.05
q = [alpha / 2, 1 - alpha / 2]
pi = simdf.quantile(q, axis=1).transpose()
pimean = np.mean(pi[0.975] - pi[0.025])
print(f"prediction interval emcee with uniform priors: {pimean:.3f} m")
print(f"PICP: {ps.stats.picp(head, pi):.3f}")